# 🔍 Demodulation Analysis

####  For each blocking card:
* Compute **Demodulation Stats**
* Compute **ASR** (Attacck Success Rate)
* Compute **Elapsed Time**

In [ ]:
# Imports
from common_imports import *

# Useful in Windwos OS
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# PATHS
DATA_PATH                 = r'../data/classic/'
RESULTS_PATH              = r'../data/results/'

# ID of the attacked MIFARE Classic
MIFARE_CLASSIC_ID   = "7e a2 42 3d a3"

# Data previously written into MIFARE Classic
MIFARE_CLASSIC_DATA = "11 22 33 44 55 66 77 88 99 77 77 77 77 77 77 77"

# List of tested Blocking Cards
BLOCKING_CARD_LIST = ['BC1','BC2','BC2','BC3','BC4','BC5','BC6','BC7','BC8','BC9','BC10','BC11','BC12','BC13','BC14']

### **1.** Compute Demodulation Stats for each Blocking Card

In [ ]:
resultsDF = pd.DataFrame(columns =  ['blocking_card_name','reader_correct', 'reader_expected', 'tag_detected', 'tag_correct', 'tag_expected', 'total_correct', 'total_expected'])

In [ ]:
for blockingCardName in BLOCKING_CARD_LIST:
      print("⚙️ START      {}".format(blockingCardName))

      # Paths
      dataPath = DATA_PATH + 'classic_{}.raw'.format(blockingCardName)
      expected_file = libnfc_file = DATA_PATH + 'expectedTxt/classic_{}.txt'.format(blockingCardName)

      # Load data
      data = load_mag(dataPath)

      # Load everything into a NfcSignal class
      start = time()
      s = NfcSignal(data, 
            expected_file = expected_file, 
            libnfc_file = libnfc_file,
            attack_mode = 0, 
            mean_samples = 0, 
            message_batch_size = 8)
      end = time()
      print("⌛ INIT TIME  {:.2f}".format(end-start))

      # Perform Demodulation
      s.start_demodulation()

      # Get demodulation stats and add blockig card blockingCardName
      print("📝 STATS: ")
      tmpDF = s.demodulation_stats()
      tmpDF['blocking_card_name'] = blockingCardName

      # Add demodulation stats to the results DF
      resultsDF = pd.concat([resultsDF, tmpDF], ignore_index=True)

In [ ]:
print(resultsDF)

### **2.** Compute ASR for each Blocking Card

In [ ]:
# List to store ASR from each blocking card
asrList = []

In [ ]:
for blockingCardName in BLOCKING_CARD_LIST:
      print("⚙️ START      {}".format(blockingCardName))

      # Paths
      dataPath = DATA_PATH + 'classic_{}.raw'.format(blockingCardName)
      expected_file = libnfc_file = DATA_PATH + 'expectedTxt/classic_{}.txt'.format(blockingCardName)

      ##################### DEMODULATION ############################
      # Load data
      data = load_mag(dataPath)

      # Load everything into a NfcSignal class
      start = time()
      s = NfcSignal(data, 
            expected_file = expected_file, 
            libnfc_file = libnfc_file,
            attack_mode = 0, 
            mean_samples = 0, 
            message_batch_size = 8)
      end = time()

      # Perform Demodulation
      s.start_demodulation()

      ################## MSG DF CREATION #############################
      # Empty list to create the DF
      msgList = []

      # Iterate through all the messages detected using NfcSignal class
      for message_start, message_end in zip(s.message_start, s.message_end):
            for m_s, m_e, m_d, m_t in zip(message_start, message_end, s.message_detected, s.message_type):
                  if m_d:
                        # Retrieve the msg
                        message = s.signal_normalized[m_s:m_e]
                        
                        # Perform Demodulation
                        hex_message = NfcSignal.perform_demodulation(
                        message, 
                        device_flag = m_t, 
                        show_plots = False,
                        hide_demodulated_data = True)
                        hex_message = hex_message.rstrip()

                        # Compute the number of bytes of each msg
                        hex_message_num_bytes = int(len(hex_message.replace(" ", ""))/2)

                        # Save results of demodulation
                        msgList.append([str(m_t),hex_message,hex_message_num_bytes])

      # Create the DF from the list
      msgDf = pd.DataFrame(msgList,columns=['msgType','msgHex','numBytes'])
      #####################################################################

      ################### DECRYPTION #######################################
      # Get possible Card Data Messages (the one composed by 18 bytes)
      encryptedCardDataList = msgDf.loc[msgDf['numBytes'] == 18]['msgHex'].tolist()

      # Get the decryption keys
      decryptionKeysFilePath = DATA_PATH + 'decryptionKeys/classic_{}_decryption_keys.txt'.format(blockingCardName)
      decryptionKeysList = open(decryptionKeysFilePath, "r").read().split("\n")[:-1]

      # Decrypted Card Data
      decryptedCardData = []

      # To count successfull decryption
      decrtyptedCount = 0

      # Try to decrypt
      for m in encryptedCardDataList:
            # Try to use all keys
            for k in decryptionKeysList:  
                  key_bytes = [int(val, 16) for val in k.split()]
                  encrypted_bytes = [int(val, 16) for val in m[:-2].split()]
                  decrypted = []

                  # XORing
                  for i in range(len(key_bytes)):
                        decrypted.append("{:02x}".format(key_bytes[i] ^ encrypted_bytes[i]))
                  decrypted = " ".join(decrypted)

                  # If MATCH with content of the card
                  if(decrypted == MIFARE_CLASSIC_DATA):
                        decrtyptedCount += 1
                        print("✅ [Attack Successfull]: {}".format(decrtyptedCount))

      # Computer ASR
      asr = float(decrtyptedCount / resultsDF.loc[resultsDF['blocking_card_name'] == blockingCardName]['total_iterations'])

      # Store ASR
      asrList.append(asr)


In [ ]:
# Add all attacck success rates to resultsDF
resultsDF['asr'] = asrList
print(resultsDF)

### **3.** Computer Rates for each blocking card

In [ ]:
# Lists to store rates
readerDemodulationRateList = []
tagDemodulationRateList = []
tagDetectionRateList = []
totalDemodulationRateList = []

# Iterate through rows
for index, row in resultsDF.iterrows():
    readerDemodulationRateList.append(round(row['reader_correct']/row['reader_expected'],2))
    tagDetectionRateList.append(round(row['tag_detected']/row['tag_expected'],2))
    tagDemodulationRateList.append(round(row['tag_correct']/row['tag_expected'],2))
    totalDemodulationRateList.append(round(row['total_correct']/row['total_expected'],2))

# Add rates to resultsDF
resultsDF['reader_demodulation_rate'] = readerDemodulationRateList
resultsDF['tag_detection_rate'] = tagDetectionRateList
resultsDF['tag_demodulation_rate'] = tagDemodulationRateList
resultsDF['total_demodulation_rate'] = totalDemodulationRateList

In [ ]:
print(resultsDF)

### **4.** Save everything into CSV

In [ ]:
# Save to CSV
resultsDF.to_csv(RESULTS_PATH + "results.csv")  

### **5.** Read files contaning elapsed time of each communication

In [ ]:
timeDF = pd.DataFrame(columns =  ['blocking_card_name','time'])

In [ ]:
for blockingCardName in BLOCKING_CARD_LIST:

      # Paths
      dataPath = DATA_PATH + 'elapsedTime/' + 'classic_time_{}.txt'.format(blockingCardName)
    
      f = open(dataPath, "r")

      time = float(f.read())

      tmpDF = {'blocking_card_name': blockingCardName, 'time': time}

      timeDF = timeDF.append(tmpDF, ignore_index = True)

In [ ]:
timeDF

### **6.** Save elapsed time into CSV

In [ ]:
# Save to CSV
timeDF.to_csv(RESULTS_PATH + "time_elapsed.csv")  